In [1]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification, get_linear_schedule_with_warmup,get_polynomial_decay_schedule_with_warmup, Adafactor
import torch.nn as nn
import logging
from lion_pytorch import Lion
from torch.utils.data import DataLoader
import GLUEforQQP
from torch.optim.lr_scheduler import LambdaLR
from functools import partial
import numpy as np
from sklearn.metrics import matthews_corrcoef
from pathlib import Path
from tqdm import tqdm
from mylion_pytorch import MyLion
import matplotlib.pyplot as plt

In [2]:
model = BertForSequenceClassification.from_pretrained('C:/Users/Xiang/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b')


Some weights of the model checkpoint at C:/Users/Xiang/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model)

In [3]:
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# tokenizer.save_pretrained("./tokenizer")
tokenizer = AutoTokenizer.from_pretrained("./tokenizer")

In [4]:

dataset_name = 'qqp'
current_path = Path.cwd().parents[0]/dataset_name
current_path.mkdir(exist_ok=True)
# lr_list = [3e-5,3e-4,3e-3]
# scheduler_list = ['no', 'linear']
# optimizer_list = ['Lion', 'AdamW','AdaFactor']
# batch_size_list = [32,64,128]
# steps = 50*1000
lr_list = [3e-4,3e-5,3e-6,3e-7]
scheduler_list = ['no', 'linear']
optimizer_list = ['Lion','AdamW','AdaFactor']
batch_size_list = [16,32,64]

lr_list = [3e-6]
scheduler_list = ['linear']
optimizer_list = ['MyLion']
batch_size_list = [64]

steps = 20*1000
report_step = 1000
accum_iter = 4

steps *= accum_iter
# steps = 100

In [5]:
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
train_dataset,valid_dataset,test_dataset = GLUEforQQP.get_torch_dataset(tokenizer, "qqp", padding="max_length", truncation=True, max_length = 350)

def constant_scheduler(
    optimizer, num_warmup_steps, num_training_steps, lr_end=1e-7, power=1.0, last_epoch=-1
):
    def lambda_func(step:int):
        return 1.

    return LambdaLR(optimizer, lambda_func, last_epoch)

def prepare(sche, opt):
    if sche == 'no':
        sches = partial(constant_scheduler)
    if sche == 'linear':
        sches = partial(get_linear_schedule_with_warmup)

    if opt == 'Lion':
        opts = partial(Lion, betas = (0.95,0.98), weight_decay = 0.01)
    if opt == 'AdaFactor':
        opts = partial(Adafactor, weight_decay = 0.001, relative_step = False, scale_parameter=False)
    if opt == 'AdamW':
        opts = partial(torch.optim.AdamW, betas = (0.9,0.99), weight_decay = 0.001)
    if opt == 'MyLion':
        opts = partial(MyLion, betas = (0.95,0.98), weight_decay = 0.01)


    return sches, opts

Using the latest cached version of the module from C:\Users\Xiang\.cache\huggingface\modules\datasets_modules\datasets\mariosasko--glue\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Sat Mar  4 15:45:32 2023) since it couldn't be found locally at mariosasko/glue., or remotely on the Hugging Face Hub.
Found cached dataset glue (C:/Users/Xiang/.cache/huggingface/datasets/mariosasko___glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at C:\Users\Xiang\.cache\huggingface\datasets\mariosasko___glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-057c3bb79acff26e.arrow
Loading cached processed dataset at C:\Users\Xiang\.cache\huggingface\datasets\mariosasko___glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-97f2e68238528875.arrow
Loading cached processed dataset at C:\Users\Xiang\.cache\huggingface\datasets\mariosasko___glue\qqp\1.0.0\dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad\cache-f0c873864ea6364f.arrow


In [6]:
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased',cache_dir  = "C:/Users/Xiang/.cache/huggingface/hub")

In [7]:


def get_log(file_name):
    logger = logging.getLogger('train')  # 设定logger的名字
    logger.setLevel(logging.INFO)  # 设定logger得等级

    ch = logging.StreamHandler()  # 输出流的hander，用与设定logger的各种信息
    ch.setLevel(logging.INFO)  # 设定输出hander的level

    fh = logging.FileHandler(file_name, mode='a')  # 文件流的hander，输出得文件名称，以及mode设置为覆盖模式
    fh.setLevel(logging.INFO)  # 设定文件hander得lever

    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)  # 两个hander设置个是，输出得信息包括，时间，信息得等级，以及message
    fh.setFormatter(formatter)
    logger.addHandler(fh)  # 将两个hander添加到我们声明的logger中去
    logger.addHandler(ch)
    return logger

In [8]:
logger = get_log('logQQP3.txt')

In [9]:
# test_loader = DataLoader(test_dataset, shuffle = False, batch_size = 32)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [10]:
def evaluate(model, dataset):
    model.eval()
    eval_loader = DataLoader(dataset, shuffle = False, batch_size = 32)
    logits = []
    labelss = []
    with torch.no_grad():
        for X in tqdm(eval_loader):
            batch = {k: v.to(device) for k, v in X.items()}
            logits.append(model(**batch).logits)
            labelss.append(batch['labels'])
        total_test = torch.concat(logits, dim = 0)
        _,predicted = torch.max(total_test,dim = 1)
        real_label =torch.concat(labelss,dim=0).cpu().numpy()
        predicted = predicted.cpu().numpy()
        metric = matthews_corrcoef(real_label, predicted)
        acc = np.mean(predicted==real_label)

    return metric, acc

In [11]:
# Training

# loss_mat = np.zeros((len(batch_size_list),len(scheduler_list), len(optimizer_list), len(lr_list),steps))

 # evaluate test metric each step
# metric_mat = np.zeros((len(batch_size_list),len(scheduler_list), len(optimizer_list), len(lr_list),steps//report_step,2))
for i,this_batch_size in enumerate(batch_size_list):
    for j,this_scheduler in enumerate(scheduler_list):
        for k,this_optimizer in enumerate(optimizer_list):
            for m, this_lr in enumerate(lr_list):
                report_loss = 0.
                loss_list = []
                metric_list = []
                acc_list = []
                train_metric_list = []
                train_acc_list = []
                model = BertForSequenceClassification.from_pretrained("C:/Users/Xiang/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b").to(device)

                train_loader = DataLoader(train_dataset, batch_size=int(this_batch_size/accum_iter))
                sche, opt = prepare(this_scheduler, this_optimizer)
                optimizer = opt(model.parameters(), lr = this_lr if this_scheduler == 'no' else this_lr*2)
                scheduler = sche(optimizer, num_warmup_steps=int(steps/(10*accum_iter)),num_training_steps=int(steps/accum_iter))
                step = 0

                metric,acc = evaluate(model,valid_dataset)
                metric_list.append(metric)
                acc_list.append(acc)

                # tmetric, tacc = evaluate(model, train_dataset)
                # train_metric_list.append(tmetric)
                # train_acc_list.append(tacc)
                # metric_mat[i,j,k,m,step//report_step - 1,0] = metric
                # metric_mat[i,j,k,m,step//report_step - 1,1] = acc

                print(f"step:{step}, matthews_corr:{metric:.6f}, Acc:{acc*100:4f}%")

                # print(f'Start training for: sche:{this_scheduler},opt:{this_optimizer},batchsize:{this_batch_size}, lr:{this_lr}')
                logger.info(f'Start training for: sche:{this_scheduler},opt:{this_optimizer},batchsize:{this_batch_size}, lr:{this_lr}')
                while True:

                    for X in train_loader:
                        model.train()

                        batch = {k: v.to(device) for k, v in X.items()}
                        loss = model(**batch).loss/accum_iter
                        report_loss+=loss.item()
                        loss.backward()
                        if (step+1)%accum_iter == 0:
                            print(f"step: {(step+1)/accum_iter}, loss:{report_loss:.8f}")

                        # loss_mat[i,j,k,m,step] = loss.item()
                            loss_list.append(report_loss)

                            optimizer.step()
                            optimizer.zero_grad()
                            scheduler.step()
                            report_loss = 0

                        step += 1

                    # valid
                        if step % (report_step*accum_iter) == 0:

                            metric,acc = evaluate(model,valid_dataset)
                            # print(i,j,k,m,step//report_step)
                            metric_list.append(metric)
                            acc_list.append(acc)
                            # tmetric, tacc = evaluate(model, train_dataset)
                            # train_metric_list.append(tmetric)
                            # train_acc_list.append(tacc)
                            # metric_mat[i,j,k,m,step//report_step - 1,0] = metric
                            # metric_mat[i,j,k,m,step//report_step - 1,1] = acc
                            # print(f"step:{step}, matthews_corr:{metric:.6f}, Acc:{acc*100:4f}%")
                            logger.info(f"step:{step/accum_iter}, matthews_corr:{metric:.6f}, Acc:{acc*100:4f}%,")

                        if step == steps:
                            break
                    if step == steps:
                        break
                file_name = dataset_name+",batchsize"+str(this_batch_size)+",scheduler"+this_scheduler+",optimizer"+str(this_optimizer)+",LR"+str(this_lr)
                np.save(current_path/(file_name+'loss.npy'),np.array(loss_list))
                np.save(current_path/(file_name+'metric.npy'),np.array(metric_list))
                np.save(current_path/(file_name+'acc.npy'),np.array(acc_list))
                # np.save(current_path/(file_name+'trainmetric.npy'),np.array(train_metric_list))
                # np.save(current_path/(file_name+'trainacc.npy'),np.array(train_acc_list))

                del model
                del optimizer
                del scheduler
                del train_loader
                torch.cuda.empty_cache()








Some weights of the model checkpoint at C:/Users/Xiang/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model)

step:0, matthews_corr:-0.087809, Acc:61.877319%
step: 1.0, loss:0.69457541
step: 2.0, loss:0.66951123
step: 3.0, loss:0.72107628
step: 4.0, loss:0.66609195
step: 5.0, loss:0.69289289
step: 6.0, loss:0.76290131
step: 7.0, loss:0.62860660
step: 8.0, loss:0.62008025
step: 9.0, loss:0.66127476
step: 10.0, loss:0.70026249
step: 11.0, loss:0.57559346
step: 12.0, loss:0.71797141
step: 13.0, loss:0.66444741
step: 14.0, loss:0.66161586
step: 15.0, loss:0.66304286
step: 16.0, loss:0.62998393
step: 17.0, loss:0.60902911
step: 18.0, loss:0.69519153
step: 19.0, loss:0.75949186
step: 20.0, loss:0.65871049
step: 21.0, loss:0.67188971
step: 22.0, loss:0.82529747
step: 23.0, loss:0.68327691
step: 24.0, loss:0.71733299
step: 25.0, loss:0.72185569
step: 26.0, loss:0.71056634
step: 27.0, loss:0.62732475
step: 28.0, loss:0.65906174
step: 29.0, loss:0.72335251
step: 30.0, loss:0.70343426
step: 31.0, loss:0.61322920
step: 32.0, loss:0.71272449
step: 33.0, loss:0.62153801
step: 34.0, loss:0.70439669
step: 35.

100%|██████████| 1264/1264 [06:57<00:00,  3.03it/s]
2023-04-02 17:43:17,661 - INFO - step:1000.0, matthews_corr:0.566281, Acc:76.126639%,


step: 1001.0, loss:0.50377335
step: 1002.0, loss:0.42402694
step: 1003.0, loss:0.43730822
step: 1004.0, loss:0.44255953
step: 1005.0, loss:0.38275766
step: 1006.0, loss:0.34251319
step: 1007.0, loss:0.34425051
step: 1008.0, loss:0.42033891
step: 1009.0, loss:0.34891225
step: 1010.0, loss:0.55529603
step: 1011.0, loss:0.49168870
step: 1012.0, loss:0.48958986
step: 1013.0, loss:0.46754875
step: 1014.0, loss:0.52816667
step: 1015.0, loss:0.53051329
step: 1016.0, loss:0.38417477
step: 1017.0, loss:0.48614062
step: 1018.0, loss:0.38896070
step: 1019.0, loss:0.46323124
step: 1020.0, loss:0.43898649
step: 1021.0, loss:0.44183081
step: 1022.0, loss:0.38663483
step: 1023.0, loss:0.31583992
step: 1024.0, loss:0.46292890
step: 1025.0, loss:0.44986677
step: 1026.0, loss:0.41096546
step: 1027.0, loss:0.41264919
step: 1028.0, loss:0.36960481
step: 1029.0, loss:0.43715471
step: 1030.0, loss:0.34854306
step: 1031.0, loss:0.48332861
step: 1032.0, loss:0.55227780
step: 1033.0, loss:0.42992652
step: 1034

100%|██████████| 1264/1264 [08:02<00:00,  2.62it/s]
2023-04-02 18:26:36,595 - INFO - step:2000.0, matthews_corr:0.669273, Acc:83.200594%,


...

100%|██████████| 1264/1264 [07:47<00:00,  2.71it/s] 
2023-04-03 06:26:38,881 - INFO - step:18000.0, matthews_corr:0.787201, Acc:90.004947%,


step: 18001.0, loss:0.15868376
step: 18002.0, loss:0.17256936
step: 18003.0, loss:0.20056924
step: 18004.0, loss:0.30804089
step: 18005.0, loss:0.13284130
step: 18006.0, loss:0.14885878
step: 18007.0, loss:0.14361217
step: 18008.0, loss:0.13350950
step: 18009.0, loss:0.07949792
step: 18010.0, loss:0.08000065
step: 18011.0, loss:0.16420801
step: 18012.0, loss:0.20162017
step: 18013.0, loss:0.27640873
step: 18014.0, loss:0.15659772
step: 18015.0, loss:0.20094265
step: 18016.0, loss:0.23863940
step: 18017.0, loss:0.12554178
step: 18018.0, loss:0.20051765
step: 18019.0, loss:0.19776305
step: 18020.0, loss:0.25087332
step: 18021.0, loss:0.17153000
step: 18022.0, loss:0.20151444
step: 18023.0, loss:0.22766722
step: 18024.0, loss:0.18460952
step: 18025.0, loss:0.12835074
step: 18026.0, loss:0.15818677
step: 18027.0, loss:0.18280732
step: 18028.0, loss:0.18382606
step: 18029.0, loss:0.24013971
step: 18030.0, loss:0.28101958
step: 18031.0, loss:0.10808058
step: 18032.0, loss:0.11328752
step: 18

100%|██████████| 1264/1264 [07:02<00:00,  2.99it/s]
2023-04-03 07:05:46,790 - INFO - step:19000.0, matthews_corr:0.785818, Acc:89.962899%,


step: 19001.0, loss:0.10336617
step: 19002.0, loss:0.14679741
step: 19003.0, loss:0.23970042
step: 19004.0, loss:0.25660980
step: 19005.0, loss:0.14857306
step: 19006.0, loss:0.21562614
step: 19007.0, loss:0.17342545
step: 19008.0, loss:0.15946268
step: 19009.0, loss:0.25000261
step: 19010.0, loss:0.10784198
step: 19011.0, loss:0.19546281
step: 19012.0, loss:0.15219539
step: 19013.0, loss:0.11110044
step: 19014.0, loss:0.20320374
step: 19015.0, loss:0.20210715
step: 19016.0, loss:0.14321972
step: 19017.0, loss:0.13002963
step: 19018.0, loss:0.17457901
step: 19019.0, loss:0.22200810
step: 19020.0, loss:0.14793122
step: 19021.0, loss:0.13272374
step: 19022.0, loss:0.28501651
step: 19023.0, loss:0.15185093
step: 19024.0, loss:0.13546640
step: 19025.0, loss:0.18059765
step: 19026.0, loss:0.12000169
step: 19027.0, loss:0.16514407
step: 19028.0, loss:0.18856536
step: 19029.0, loss:0.13998816
step: 19030.0, loss:0.14265320
step: 19031.0, loss:0.20459912
step: 19032.0, loss:0.11060892
step: 19

100%|██████████| 1264/1264 [07:02<00:00,  2.99it/s]
2023-04-03 07:44:52,283 - INFO - step:20000.0, matthews_corr:0.784825, Acc:89.883750%,
